In [1]:
import pandas as pd

df = pd.read_csv("/kaggle/input/toxic-comment/train.csv")
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))

In [3]:
def clean_text(text):
    if not text:
        return ""
    text = str(text)    
    text = re.sub(r'[\n\r\t\f\v]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    words = text.split()
    filtered_words = [word for word in words if word and word not in stop_words and len(word) > 1]
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

df['comment_text'] = df['comment_text'].apply(clean_text)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matches background colour im seemingly st...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really trying edit war guy constant...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time asking view completely contradicts...,0,0,0,0,0,0
159567,ffea4adeee384e90,ashamed horrible thing put talk page,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres actual article prostitution...,0,0,0,0,0,0
159569,fff125370e4aaaf3,looks like actually put speedy first version d...,0,0,0,0,0,0


In [4]:
df = df.dropna().drop(['id'], axis=1)
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,daww matches background colour im seemingly st...,0,0,0,0,0,0
2,hey man im really trying edit war guy constant...,0,0,0,0,0,0
3,cant make real suggestions improvement wondere...,0,0,0,0,0,0
4,sir hero chance remember page thats,0,0,0,0,0,0
...,...,...,...,...,...,...,...
159566,second time asking view completely contradicts...,0,0,0,0,0,0
159567,ashamed horrible thing put talk page,0,0,0,0,0,0
159568,spitzer umm theres actual article prostitution...,0,0,0,0,0,0
159569,looks like actually put speedy first version d...,0,0,0,0,0,0


In [5]:
from collections import Counter

def build_vocab(texts, max_vocab=50000):
    counter = Counter()
    for line in texts:
        counter.update(line.split())
    most_common = counter.most_common(max_vocab - 2)
    vocab = {"<PAD>": 0, "<UNK>": 1}
    vocab.update({word: i+2 for i, (word, _) in enumerate(most_common)})
    return vocab

def encode(text, vocab):
    return [vocab.get(w, vocab["<UNK>"]) for w in text.split()]

vocab = build_vocab(df['comment_text'].tolist(), max_vocab=216477)

In [6]:
from sklearn.model_selection import train_test_split

y = df.drop(['comment_text'], axis = 1).values
X_train, X_test, y_train, y_test = train_test_split(df['comment_text'], y, test_size=0.2, random_state=42, stratify=y[:, 0])

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
y_train = torch.tensor(y_train)
pos_counts = y_train.sum(dim=0).float()
neg_counts = len(y_train) - pos_counts
pos_weights = neg_counts / pos_counts
pos_weights = pos_weights.to(device) 
pos_weights

tensor([  9.4337,  98.4206,  17.9429, 334.0551,  19.3048, 113.3871],
       device='cuda:0')

In [9]:
!pip install googletrans==4.0.0rc1 tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=6e410d52ad2ffc2918a915e57c6721e50e71de796678ba77920dce9f13ec56e5
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [10]:
from googletrans import Translator
import random, time, torch
from tqdm import tqdm

translator = Translator()
langs = ['es', 'fr', 'de']

for i in range(y_train.shape[1]):
    if pos_weights[i] < 200: continue
    pos = torch.where(y_train[:, i] == 1)[0]
    needed = (y_train[:, i] == 0).sum() - len(pos)
    if needed <= 0: continue
    
    new_texts, new_labels = [], []
    for _ in tqdm(range(min(needed, 2 * len(pos))), desc=f'Column {i}'):
        idx = random.choice(pos.tolist())
        try:
            t1 = translator.translate(X_train[idx], dest=random.choice(langs))
            time.sleep(0.1)
            t2 = translator.translate(t1.text, dest='en')
            new_texts.append(t2.text)
        except: new_texts.append(X_train.iloc[idx])
        new_labels.append(y_train[idx])
    
    X_train = X_train.tolist() + new_texts
    y_train = torch.cat([y_train, torch.stack(new_labels)])

s = torch.randperm(len(X_train))
X_train, y_train = [X_train[i] for i in s], y_train[s]

Column 3: 100%|██████████| 762/762 [14:26<00:00,  1.14s/it]


In [11]:
y_train = torch.tensor(y_train)
pos_counts = y_train.sum(dim=0).float()
neg_counts = len(y_train) - pos_counts
neg_counts / pos_counts

/tmp/ipykernel_19/354795991.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train)


tensor([  8.9203,  87.2598,  16.8012, 111.3517,  17.9771, 100.1165])

In [12]:
X_train_encode = [encode(sent, vocab) for sent in X_train]
X_test_encode = [encode(sent, vocab) for sent in X_test]

In [13]:
class ToxicDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), self.labels[idx]

def collate_fn(batch):
    texts, labels = zip(*batch)
    lengths = [len(x) for x in texts]
    padded = pad_sequence(texts, batch_first=True, padding_value=0)
    return padded, torch.stack(labels)

train_ds = ToxicDataset(X_train_encode, y_train)
val_ds = ToxicDataset(X_test_encode, y_test)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, collate_fn=collate_fn)

/tmp/ipykernel_19/878285299.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)


In [14]:
import torch
import torch.nn as nn

class SimpleBiLSTMToxicClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=100, hidden_dim=64, dropout=0.3):
        super(SimpleBiLSTMToxicClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, 
                           bidirectional=True, dropout=dropout)
        

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, 6)
        
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        
        lstm_out, (h_n, c_n) = self.lstm(embedded)
        
        hidden = torch.cat((h_n[-2], h_n[-1]), dim=1) 
        
        output = self.dropout(hidden)
        logits = self.fc(output)
        
        return torch.sigmoid(logits)


In [15]:
y_train = torch.tensor(y_train)
pos_counts = y_train.sum(dim=0).float()
neg_counts = len(y_train) - pos_counts
pos_weights = neg_counts / pos_counts
pos_weights = pos_weights.to(device) 
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

/tmp/ipykernel_19/3491468082.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train)


In [16]:
pos_weights

tensor([  8.9203,  87.2598,  16.8012, 111.3517,  17.9771, 100.1165],
       device='cuda:0')

In [17]:
from tqdm import tqdm


model = SimpleBiLSTMToxicClassifier(vocab_size=len(vocab)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

num_epochs = 20
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=False)

    for batch_x, batch_y in train_loop:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_loop.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)

    # Validation phase
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", leave=False)
        for batch_x, batch_y in val_loop:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_val_loss += loss.item()
            val_loop.set_postfix(val_loss=loss.item())

    avg_val_loss = total_val_loss / len(val_loader)
    # Epoch summary
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Epoch 1/20 | Train Loss: 1.1645 | Val Loss: 1.0541


Epoch 2/20 | Train Loss: 1.0919 | Val Loss: 1.0232


Epoch 3/20 | Train Loss: 1.0621 | Val Loss: 1.0019


Epoch 4/20 | Train Loss: 1.0470 | Val Loss: 0.9942


...

Epoch 16/20 | Train Loss: 1.0270 | Val Loss: 0.9843


Epoch 17/20 | Train Loss: 1.1055 | Val Loss: 1.5135


Epoch 18/20 | Train Loss: 1.2273 | Val Loss: 1.5115


Epoch 19/20 | Train Loss: 1.3828 | Val Loss: 0.9923


Epoch 20/20 | Train Loss: 1.0874 | Val Loss: 0.9891


In [18]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch_x, batch_y in val_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x).cpu()
        preds = (outputs > 0.5).float()
        all_preds.append(preds)
        all_labels.append(batch_y)

y_pred = torch.cat(all_preds).numpy()
y_true = torch.cat(all_labels).numpy()

print("F1 Score (micro):", f1_score(y_true, y_pred, average='micro'))
print("Accuracy Score: ", accuracy_score(y_true, y_pred))
print("AUR ROC: ", roc_auc_score(y_true, y_pred))
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print("Classification Report: \n",classification_report(y_true, y_pred, target_names=labels))

F1 Score (micro): 0.5086316863678884
Accuracy Score:  0.8721290929030237
AUR ROC:  0.8999309169256281
Classification Report: 
                precision    recall  f1-score   support

        toxic       0.69      0.79      0.74      3059
 severe_toxic       0.13      0.96      0.24       311
      obscene       0.54      0.90      0.67      1710
       threat       0.03      0.72      0.07        97
       insult       0.47      0.87      0.61      1590
identity_hate       0.09      0.89      0.16       289

    micro avg       0.36      0.85      0.51      7056
    macro avg       0.33      0.86      0.41      7056
 weighted avg       0.55      0.85      0.64      7056
  samples avg       0.04      0.08      0.05      7056



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
